# email 

In [ ]:
19157521820@163.com

18285998868@163.com

3124568493@qq.com

kefu1820@gmail.com

# add brtfs part to fstab

In [ ]:
%%bash
# echo "Adding /data partition"
# lsblk -f
# sudo mkdir -p /data && echo "UUID=180161ab-546d-4dfe-969e-e1cf9938195e /data btrfs defaults,compress=zstd,noatime 0 0" | sudo tee -a /etc/fstab

# systemctl daemon-reload
# sudo mount -a
# df -h | grep /data
# sudo chown $USER:$USER /data
# sudo chmod 755 /data


# software

# Uninstall

In [ ]:
%%bash
sudo pacman -Rns code --noconfirm

# delete the folder
folder_path=$(pwd)/bin/vscode
rm -rf $folder_path
# delete the pth in ~/.zshrc
sed -i '/^export.*vscode/d' ~/.zshrc

